## Part 1: Data Loading and Initial Exploration (15 pts)

The data for this project is stored in the `data` folder in your repositories, in the `SOTU.csv` file. The data file is structured as a CSV with columns for president name, speech text, year, and word count in the speech. 

In this section you will:

1. Import the data into a pandas dataframe
2. Perform exploratory data analysis (EDA) including specifically:
- Analyze the number of speeches per president
- Analyze the number of speeches per year
- Analyze the word count distribution
- Analyze the word count distribution accross years using a rug plot
- Analyze the average word count per president
3. Write commentary on your findings

First, create the `conda` environment with the provided yaml file. Note, it's not unusual for it to take ~15 minutes for the environment to fully install.

## Read Data


In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
#conda install -c conda-forge spacy 
#python -m spacy download en_core_web_sm
plt.style.use('seaborn-v0_8-dark') 
sou = pd.read_csv("data/SOTU.csv")
sou

,President,Year,Text,Word Count
0,Joseph R. Biden,2024.0,"\n[Before speaking, the President presented hi...",8003
1,Joseph R. Biden,2023.0,\nThe President. Mr. Speaker——\n[At this point...,8978
2,Joseph R. Biden,2022.0,"\nThe President. Thank you all very, very much...",7539
3,Joseph R. Biden,2021.0,\nThe President. Thank you. Thank you. Thank y...,7734
4,Donald J. Trump,2020.0,\nThe President. Thank you very much. Thank yo...,6169
...,...,...,...,...
241,George Washington,1791.0,\nFellow-Citizens of the Senate and House of R...,2264
242,George Washington,1790.0,\nFellow-Citizens of the Senate and House of R...,1069
243,George Washington,1790.0,\nFellow-Citizens of the Senate and House of R...,1069
244,George Washington,1790.0,\nFellow-Citizens of the Senate and House of R...,1069


#### Number of Speeches per President

In [3]:
order = (
    sou.groupby('President')['Year']
    .max()
    .sort_values(ascending = False)
    .index
    .tolist()
)

tmpsou = sou
tmpsou['President'] = pd.Categorical(sou['President'], categories = order, ordered = True)

plt.figure(figsize = (15.3,5))
plt.ylabel("count")
tmpsou['President'].value_counts().loc[order].plot(kind = 'bar')

plt.savefig("outputs/part1outputs/president_counts.png", dpi=300)  
plt.show()

Total speeches in full dataset: 246
Speeches from 2000 and later: 25


,year,president
0,2024.0,Joseph R. Biden
1,2023.0,Joseph R. Biden
2,2022.0,Joseph R. Biden
3,2021.0,Joseph R. Biden
4,2020.0,Donald J. Trump


#### Number of Speeches per Year


In this section, I use spaCy to process the State of the Union speeches.

- **Token**: a single word-like unit from the text (for example, "running", "people").
- **Lemma**: the base form of a word (for example, "running" → "run", "better" → "good").
- **Stop word**: a very common word that usually does not add much meaning, like "the", "and", "of".
- **Punctuation**: symbols like ".", ",", "!", "?", etc.

For this analysis, I remove stop words and punctuation, and I work mostly with **lemmas**.  
Using lemmas helps to group different forms of the same word together (for example, "runs", "running", "ran" all become "run"), which makes the frequency counts and models less noisy.


In [6]:
# Hint - Use value counts and sort by years
counts = sou.groupby('Year').size()
plt.ylabel("count")
counts.plot(kind = 'line')
plt.savefig("outputs/part1outputs/number_of_speeches_per_year.png", dpi=300)  
plt.show()


#### Word Count Distribution 

In [ ]:
wordCounts = []
nlp = spacy.load('en_core_web_sm')
sou_subset = sou['Text']
for i in range(len(sou_subset)):
    count = 0
    tokens = nlp(sou_subset[i])
    for t in tokens:
        if t.is_punct == False and t.is_space == False:
            count += 1
    wordCounts.append(count)

In [ ]:
plt.hist(wordCounts, bins = 19,edgecolor='black')
plt.xlabel("word count")
plt.ylabel("count")
plt.savefig("outputs/part1outputs/word_counts.png", dpi=300)  
plt.show()

#### Word Count Distribution over Year

In [11]:
def count_tokens(text):
    count = 0
    tokens = nlp(text)
    for t in tokens:
        if t.is_punct == False and t.is_space == False:
            count += 1
    return count

sou['token_count'] = sou['Text'].apply(count_tokens)

year_Count = sou.groupby('Year')['token_count'].sum().reset_index()

Top 20 tokens:
[('government', 7462), ('states', 6837), ('congress', 5403), ('united', 5113), ('people', 4458), ('year', 4394), ('country', 3724), ('great', 3479), ('public', 3356), ('new', 3228), ('time', 3128), ('american', 3009), ('war', 2877), ('world', 2771), ('years', 2614), ('law', 2311), ('nation', 2300), ('national', 2291), ('state', 2253), ('present', 2222)]

Top 20 lemmas:
[('government', 8293), ('year', 7008), ('states', 6517), ('congress', 5408), ('united', 5050), ('country', 4790), ('people', 4651), ('great', 4447), ('nation', 3985), ('law', 3737), ('time', 3603), ('public', 3356), ('new', 3240), ('war', 3037), ('american', 3022), ('power', 2977), ('work', 2919), ('world', 2772), ('state', 2770), ('increase', 2759)]


In [ ]:
plt.ylabel("year")
plt.xlabel("count")
plt.scatter(year_Count['token_count'],year_Count['Year'])

plt.savefig("outputs/part1outputs/word_dist_per_year.png", dpi=300)  
plt.show()

#### Word Count Distribution per President

In [12]:
president_Count = sou.groupby('President')['token_count'].sum().reset_index()
president_Count.plot(
    kind='bar',
    x='President',
    y='token_count',
    figsize=(12,4)
)

plt.savefig("outputs/part1outputs/count_per_president.png", dpi=300)  
plt.show()

,year,president,lemmas_str
0,2024.0,Joseph R. Biden,speak president present prepared remark speake...
1,2023.0,Joseph R. Biden,president speaker point president turn face au...
2,2022.0,Joseph R. Biden,president thank thank thank madam speaker mada...
3,2021.0,Joseph R. Biden,president thank thank thank good mitch chuck u...
4,2020.0,Donald J. Trump,president thank thank thank madam speaker vice...
